In [13]:
import pandas as pd
from datetime import datetime, timedelta
import requests
import time

In [14]:
pd.set_option('display.max_columns', None)

- - - 

# Trapist

In [15]:
todas_recargas=pd.DataFrame()

In [16]:
start_date = '2024-03-08'
end_date = '2024-03-10'
id_loja='3123a77b-2730-4b44-ab69-24bba879c56b'


token = "hlqFYMUrmLX9NGTd07jC5ZjxLRKsOfqh"


#RECARGAS

api_url = "https://api.zigcore.com.br/integration/erp/recharges"
token = "hlqFYMUrmLX9NGTd07jC5ZjxLRKsOfqh"

params = {
    'dtinicio': start_date,
    'dtfim': end_date,
    'loja': id_loja
}


headers = {
    "Authorization": f"{token}",
    "Content-Type": "application/json"
}


response_recarga = requests.get(api_url, headers=headers, params=params)

recarga=pd.DataFrame(response_recarga.json())

recarga=recarga[["date","author","totalValue","userDocument","userDocumentType","userName"]]

#Corrigindo os valores
recarga["totalValue"]= recarga["totalValue"]/100

#Valores das datas
recarga["date"]=pd.to_datetime(recarga["date"])
recarga["Dia + Hora"]=recarga["date"].dt.strftime('%Y-%m-%d %H')

todas_recargas=pd.concat([todas_recargas,recarga], ignore_index=True)


In [17]:
todas_recargas["Dia Hora"]=todas_recargas["date"].dt.strftime('%Y-%m-%d %H:00:00')

In [18]:
todas_recargas

,date,author,totalValue,userDocument,userDocumentType,userName,Dia + Hora,Dia Hora
0,2024-03-09 18:14:57.734,CAIXA 38,13.0,50374464812,cpf,matheus,2024-03-09 18,2024-03-09 18:00:00
1,2024-03-09 18:15:30.730,CAIXA 37,50.0,43772406840,cpf,danilo,2024-03-09 18,2024-03-09 18:00:00
2,2024-03-09 18:15:51.451,CAIXA 8,55.0,48046697877,cpf,Caio,2024-03-09 18,2024-03-09 18:00:00
3,2024-03-09 18:47:26.022,CAIXA 19,200.0,55770139846,cpf,olyver,2024-03-09 18,2024-03-09 18:00:00
4,2024-03-09 18:59:09.811,CAIXA 51,21.0,51567473881,cpf,Nathan,2024-03-09 18,2024-03-09 18:00:00
...,...,...,...,...,...,...,...,...
8605,2024-03-10 04:09:41.167,CAIXA 25,10.0,36308616877,cpf,vitor,2024-03-10 04,2024-03-10 04:00:00
8606,2024-03-10 04:31:49.353,CAIXA 6,30.0,40867258861,cpf,Juan,2024-03-10 04,2024-03-10 04:00:00
8607,2024-03-10 04:33:47.524,CAIXA 50,5.0,48514999818,cpf,Isabella El Dorr da Silva,2024-03-10 04,2024-03-10 04:00:00
8608,2024-03-10 04:52:33.385,CAIXA 38,100.0,39756532807,cpf,Guilherme secco,2024-03-10 04,2024-03-10 04:00:00


In [19]:
todas_recargas.rename(columns={
    "author":"nome_ponto",
    "totalValue":"valor",

}, inplace=True)

In [31]:
todas_recargas["nome_evento"]="TRAPIST - 09/03/2024"

- - - 

In [20]:
ID_EVENTOS=[
    15740, #Tardezinha Campinas 2023
    16092, #TADEZINHA CAMPINAS 2023 LOJA
    17990, #Tardezinha RB 2023
    19364, #Trapist
    19404, #Trapist 2024 
    21031, #Soweto Jul24
    21032, # ME LEVA
    21033 #NUMANICE TOUR 3  CAMPINAS


]

DATA_INICIO=[
    "2023-07-15", #Tardezinha Campinas 2023
    "2023-07-15", #TADEZINHA CAMPINAS 2023 LOJA
    "2023-12-02", #Tardezinha RB 2023
    "2024-03-09", #Trapist
    "2024-03-09", #Trapist 2024 
    "2024-07-27", #Soweto Jul24
    "2024-10-19", # ME LEVA
    "2024-10-12" #NUMANICE TOUR 3  CAMPINAS


]




#Definir a Dataframe de produtos (df_produtos)
df_pagamentos=pd.DataFrame()

#####################################################Geração do Token de acesso

for i in range(len(ID_EVENTOS)):
    #api_url = "http://devapp.netpdv.info:5566/api/relatorios/GeraToken"
    api_url= "https://app.netpdv.com:5566/api/relatorios/GeraToken"

    #tardezinha Campinas 15740
    #tardezinha RB  17990


    headers = {
        
        "Content-Type": "text/plain",
        "User-Agent": "PostmanRuntime/7.24.0",
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
        "token_parceiro": "BCED47DAE6",
        "Content-type": "application/json"
        

    }

    #Tardezinha Campinas
    body = {
        'codigo_evento': ID_EVENTOS[i]
    }




    response_token = requests.post(api_url, headers=headers, json=body)
    token_json=response_token.json()
    token=token_json['dados']['token_acesso']
    






    ################################################# Fazendo as requisições #####################


    api_url="https://app.netpdv.com:5566/api/relatorios/ListaAdquirencia"


    #deixar setado para inicio às 12hrs e fim às 06hrs do dia seguinte
    #data início sempre deve ser igual à data fim. O código pega as 12hrs da data início e vai até as 06hrs do dia seguinte
    data_inicio=DATA_INICIO[i]
    data_fim=DATA_INICIO[i]

    horario_inicio = "12:00:00"
    horario_fim="13:00:00"




    

    #Loop para pegar as transações (Preciso adicionar um for loop antes para cada evento)
    #ATENÇÃO: Talvez seja necessário gerar outro token
    print(ID_EVENTOS[i])
    while horario_fim!="07:00:00":

        body={
            "data_inicio":data_inicio+" "+horario_inicio,
            "data_fim":data_fim+" "+horario_fim,
        }


        headers = {
            "token_acesso":token
        }

        print(horario_fim)

        ##############################################################
        #Atualizando os horários para o próximo loop
        horario_inicio=pd.to_datetime(horario_inicio, format='%H:%M:%S')
        horario_inicio +=timedelta(hours=1)
        horario_inicio=horario_inicio.strftime('%H:%M:%S')

        horario_fim=pd.to_datetime(horario_fim, format='%H:%M:%S')
        horario_fim +=timedelta(hours=1)
        horario_fim=horario_fim.strftime('%H:%M:%S')



        if horario_fim=="00:00:00":
            data_fim=pd.to_datetime(data_fim, format='%Y-%m-%d')
            data_fim += timedelta(days=1)
            data_fim=data_fim.strftime('%Y-%m-%d')

        if horario_inicio=="00:00:00":
            data_inicio=pd.to_datetime(data_inicio, format='%Y-%m-%d')
            data_inicio += timedelta(days=1)
            data_inicio=data_inicio.strftime('%Y-%m-%d')

        #############################################



        time.sleep(5)
        response_pagamentos = requests.post(api_url, headers=headers, json=body)
        pagamentos_json=response_pagamentos.json()

        pagamentos = pagamentos_json["dados"]["pagamentos"]
        df_pagamentos_horario=pd.DataFrame(pagamentos)

        df_pagamentos=pd.concat([df_pagamentos,df_pagamentos_horario], ignore_index=True)


        df_pagamentos["data_hora_realizacao"]=pd.to_datetime(df_pagamentos["data_hora_realizacao"])






df_pagamentos

15740
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
06:00:00
16092
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
06:00:00
17990
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
06:00:00
19364
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
06:00:00
19404
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
06:00:00
21031
13:00:00
14:00:00
15:00:00
16:00:00
17:00:00
18:00:00
19:00:00
20:00:00
21:00:00
22:00:00
23:00:00
00:00:00
01:00:00
02:00:00
03:00:00
04:00:00
05:00:00
0

,pagamento_id,codigo_evento,nome_evento,data_hora_realizacao,data_hora_adquirente,codigo_ponto,nome_ponto,operador,terminal,adquirente,operacao,valor,status,cod_forma_pagamento,forma_pagamento,bandeira,ultimos_digitos,pagamento_adquirente_id,pagamento_adquirente_code,nsu,codigo_autorizacao
0,B90DA43B-EBB3-484A-B95C-459C84A91D51,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 12:52:40,2023-07-15T12:52:32,000043,VIP.CM.DINHEIRO.M03,00000776,860614060808331,PagSeguro,VENDA,200.0,Efetivada,6,Cartao Debito,MAESTRO,7379,ED413C6A3D,C688046C0ADB477AACD3B255942A1FB2,071517118546,309981
1,84DE4A3F-4291-455F-BBDA-8E10C499695C,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 12:59:50,2023-07-15T12:59:43,000009,PISTA.CM.DINHEIRO.M01,00000136,860614061870611,PagSeguro,VENDA,43.0,Efetivada,7,Cartao Credito,MASTERCARD,0425,87321DA510,FEF579E0F3AB4877884497E593AFBFF6,071584832192,560754
2,B6CD1C93-E77D-4DC3-BB5C-365061EB88CD,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 13:42:47,2023-07-15T13:42:40,000009,PISTA.CM.DINHEIRO.M01,00000110,867732062630744,PagSeguro,VENDA,40.0,Efetivada,6,Cartao Debito,VISA_ELECTRON,7703,77715C5011,213D795CADE44588ADB34C49DED31C33,319616187068,NEU9DM
3,42D4835B-579D-41E7-A986-7311A9DA6DDE,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 13:42:57,2023-07-15T13:42:49,000045,VIP.CAIXA FIXO.C02,00000916,860614061509433,PagSeguro,VENDA,877.0,Efetivada,7,Cartao Credito,MASTERCARD,2908,3DFF1D2D08,D942723C2D1146ED8670ED69796A142A,071581100932,077975
4,399DFBA3-CFD3-49F5-B911-BAD8C4D3CB6C,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 13:42:58,2023-07-15T13:42:48,000009,PISTA.CM.DINHEIRO.M01,00000143,860614061645146,PagSeguro,VENDA,190.0,Efetivada,6,Cartao Debito,VISA_ELECTRON,0764,BD4FBC90C3,20FDEC6F62B14E578CB9DDB60C188FBA,319616055431,225123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71855,8779E48A-5697-48D6-B548-8A26869555AC,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:27:50,2024-10-13T00:28:05,000004,CM.FRONT DINHEIRO,00000013,868731070703112,PinBank,VENDA,65.0,Efetivada,7,Cartao Credito,MASTERCARD,9733,000024905493,115181568,164,798813
71856,D20B5A3F-0EE3-4446-B686-F56939630605,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:29:05,2024-10-13T00:29:12,000005,CM.FRONT,00000099,864793076190359,PinBank,VENDA,23.0,Efetivada,7,Cartao Credito,MASTERCARD,1803,000024905603,115181680,343,859123
71857,0E33B657-6F9B-4B59-970A-A33852B4D167,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:29:37,2024-10-13T00:29:48,000004,CM.FRONT DINHEIRO,00000013,868731070703112,PinBank,VENDA,40.0,Efetivada,6,Cartao Debito,ELO,6279,000024905669,115181748,165,309540
71858,76C85B6E-8E10-4E81-9EC9-DDD733A5A432,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:30:05,2024-10-13T00:30:58,000004,CM.FRONT DINHEIRO,00000030,864793074119087,PinBank,VENDA,76.0,Efetivada,7,Cartao Credito,MASTERCARD,6841,000024905785,115181865,160,305971


In [21]:
df_pagamentos["Dia Hora"]=df_pagamentos["data_hora_realizacao"].dt.strftime('%Y-%m-%d %H:00:00')
df_pagamentos

,pagamento_id,codigo_evento,nome_evento,data_hora_realizacao,data_hora_adquirente,codigo_ponto,nome_ponto,operador,terminal,adquirente,operacao,valor,status,cod_forma_pagamento,forma_pagamento,bandeira,ultimos_digitos,pagamento_adquirente_id,pagamento_adquirente_code,nsu,codigo_autorizacao,Dia Hora
0,B90DA43B-EBB3-484A-B95C-459C84A91D51,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 12:52:40,2023-07-15T12:52:32,000043,VIP.CM.DINHEIRO.M03,00000776,860614060808331,PagSeguro,VENDA,200.0,Efetivada,6,Cartao Debito,MAESTRO,7379,ED413C6A3D,C688046C0ADB477AACD3B255942A1FB2,071517118546,309981,2023-07-15 12:00:00
1,84DE4A3F-4291-455F-BBDA-8E10C499695C,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 12:59:50,2023-07-15T12:59:43,000009,PISTA.CM.DINHEIRO.M01,00000136,860614061870611,PagSeguro,VENDA,43.0,Efetivada,7,Cartao Credito,MASTERCARD,0425,87321DA510,FEF579E0F3AB4877884497E593AFBFF6,071584832192,560754,2023-07-15 12:00:00
2,B6CD1C93-E77D-4DC3-BB5C-365061EB88CD,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 13:42:47,2023-07-15T13:42:40,000009,PISTA.CM.DINHEIRO.M01,00000110,867732062630744,PagSeguro,VENDA,40.0,Efetivada,6,Cartao Debito,VISA_ELECTRON,7703,77715C5011,213D795CADE44588ADB34C49DED31C33,319616187068,NEU9DM,2023-07-15 13:00:00
3,42D4835B-579D-41E7-A986-7311A9DA6DDE,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 13:42:57,2023-07-15T13:42:49,000045,VIP.CAIXA FIXO.C02,00000916,860614061509433,PagSeguro,VENDA,877.0,Efetivada,7,Cartao Credito,MASTERCARD,2908,3DFF1D2D08,D942723C2D1146ED8670ED69796A142A,071581100932,077975,2023-07-15 13:00:00
4,399DFBA3-CFD3-49F5-B911-BAD8C4D3CB6C,15740,TARDEZINHA CAMPINAS 2023,2023-07-15 13:42:58,2023-07-15T13:42:48,000009,PISTA.CM.DINHEIRO.M01,00000143,860614061645146,PagSeguro,VENDA,190.0,Efetivada,6,Cartao Debito,VISA_ELECTRON,0764,BD4FBC90C3,20FDEC6F62B14E578CB9DDB60C188FBA,319616055431,225123,2023-07-15 13:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71855,8779E48A-5697-48D6-B548-8A26869555AC,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:27:50,2024-10-13T00:28:05,000004,CM.FRONT DINHEIRO,00000013,868731070703112,PinBank,VENDA,65.0,Efetivada,7,Cartao Credito,MASTERCARD,9733,000024905493,115181568,164,798813,2024-10-13 00:00:00
71856,D20B5A3F-0EE3-4446-B686-F56939630605,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:29:05,2024-10-13T00:29:12,000005,CM.FRONT,00000099,864793076190359,PinBank,VENDA,23.0,Efetivada,7,Cartao Credito,MASTERCARD,1803,000024905603,115181680,343,859123,2024-10-13 00:00:00
71857,0E33B657-6F9B-4B59-970A-A33852B4D167,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:29:37,2024-10-13T00:29:48,000004,CM.FRONT DINHEIRO,00000013,868731070703112,PinBank,VENDA,40.0,Efetivada,6,Cartao Debito,ELO,6279,000024905669,115181748,165,309540,2024-10-13 00:00:00
71858,76C85B6E-8E10-4E81-9EC9-DDD733A5A432,21033,NUMANICE TOUR 3 CAMPINAS,2024-10-13 00:30:05,2024-10-13T00:30:58,000004,CM.FRONT DINHEIRO,00000030,864793074119087,PinBank,VENDA,76.0,Efetivada,7,Cartao Credito,MASTERCARD,6841,000024905785,115181865,160,305971,2024-10-13 00:00:00


In [32]:
df_pagamentos_concatenado=pd.concat([df_pagamentos,todas_recargas], ignore_index=True)

In [33]:
df_pagamentos_concatenado.to_excel("financeiro_zig.xlsx", index=False, sheet_name="Financeiro")

- - - 

In [29]:
df_pagamentos_concatenado

,pagamento_id,codigo_evento,nome_evento,data_hora_realizacao,data_hora_adquirente,codigo_ponto,nome_ponto,operador,terminal,adquirente,operacao,valor,status,cod_forma_pagamento,forma_pagamento,bandeira,ultimos_digitos,pagamento_adquirente_id,pagamento_adquirente_code,nsu,codigo_autorizacao,Dia Hora,date,userDocument,userDocumentType,userName,Dia + Hora
0,B90DA43B-EBB3-484A-B95C-459C84A91D51,15740.0,TARDEZINHA CAMPINAS 2023,2023-07-15 12:52:40,2023-07-15T12:52:32,000043,VIP.CM.DINHEIRO.M03,00000776,860614060808331,PagSeguro,VENDA,200.0,Efetivada,6.0,Cartao Debito,MAESTRO,7379,ED413C6A3D,C688046C0ADB477AACD3B255942A1FB2,071517118546,309981,2023-07-15 12:00:00,NaT,NaN,NaN,NaN,NaN
1,84DE4A3F-4291-455F-BBDA-8E10C499695C,15740.0,TARDEZINHA CAMPINAS 2023,2023-07-15 12:59:50,2023-07-15T12:59:43,000009,PISTA.CM.DINHEIRO.M01,00000136,860614061870611,PagSeguro,VENDA,43.0,Efetivada,7.0,Cartao Credito,MASTERCARD,0425,87321DA510,FEF579E0F3AB4877884497E593AFBFF6,071584832192,560754,2023-07-15 12:00:00,NaT,NaN,NaN,NaN,NaN
2,B6CD1C93-E77D-4DC3-BB5C-365061EB88CD,15740.0,TARDEZINHA CAMPINAS 2023,2023-07-15 13:42:47,2023-07-15T13:42:40,000009,PISTA.CM.DINHEIRO.M01,00000110,867732062630744,PagSeguro,VENDA,40.0,Efetivada,6.0,Cartao Debito,VISA_ELECTRON,7703,77715C5011,213D795CADE44588ADB34C49DED31C33,319616187068,NEU9DM,2023-07-15 13:00:00,NaT,NaN,NaN,NaN,NaN
3,42D4835B-579D-41E7-A986-7311A9DA6DDE,15740.0,TARDEZINHA CAMPINAS 2023,2023-07-15 13:42:57,2023-07-15T13:42:49,000045,VIP.CAIXA FIXO.C02,00000916,860614061509433,PagSeguro,VENDA,877.0,Efetivada,7.0,Cartao Credito,MASTERCARD,2908,3DFF1D2D08,D942723C2D1146ED8670ED69796A142A,071581100932,077975,2023-07-15 13:00:00,NaT,NaN,NaN,NaN,NaN
4,399DFBA3-CFD3-49F5-B911-BAD8C4D3CB6C,15740.0,TARDEZINHA CAMPINAS 2023,2023-07-15 13:42:58,2023-07-15T13:42:48,000009,PISTA.CM.DINHEIRO.M01,00000143,860614061645146,PagSeguro,VENDA,190.0,Efetivada,6.0,Cartao Debito,VISA_ELECTRON,0764,BD4FBC90C3,20FDEC6F62B14E578CB9DDB60C188FBA,319616055431,225123,2023-07-15 13:00:00,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80465,NaN,NaN,Trapist,NaT,NaN,NaN,CAIXA 25,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-10 04:00:00,2024-03-10 04:09:41.167,36308616877,cpf,vitor,2024-03-10 04
80466,NaN,NaN,Trapist,NaT,NaN,NaN,CAIXA 6,NaN,NaN,NaN,NaN,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-10 04:00:00,2024-03-10 04:31:49.353,40867258861,cpf,Juan,2024-03-10 04
80467,NaN,NaN,Trapist,NaT,NaN,NaN,CAIXA 50,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-10 04:00:00,2024-03-10 04:33:47.524,48514999818,cpf,Isabella El Dorr da Silva,2024-03-10 04
80468,NaN,NaN,Trapist,NaT,NaN,NaN,CAIXA 38,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-03-10 04:00:00,2024-03-10 04:52:33.385,39756532807,cpf,Guilherme secco,2024-03-10 04


In [26]:
df_pagamentos_concatenado["nome_evento"].unique()

array(['TARDEZINHA CAMPINAS 2023', 'TADEZINHA CAMPINAS 2023 LOJA',
       'TARDEZINHA RB 2023', 'SOWETO JUL24', 'NUMANICE TOUR 3  CAMPINAS',
       nan], dtype=object)